In [4]:
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
from pandas_datareader import data as web
import os 
import pickle
import numpy as np
from collections import Counter
from nsepy import get_history
from finta import TA
import copy
from tqdm import tqdm



style.use('ggplot')
%matplotlib qt

In [5]:
df = get_history(symbol="HDFCBANK", start=dt.date(2019,1,1), end = dt.date.today())
#print(df.tail)
end_date = df.index[-1]
df_extracted = pd.read_csv('stocks_dfs/{}.csv'.format("HDFCBANK"), header= 0)
df_extracted.index = df_extracted.Date
extracted_date = dt.datetime.strptime(df_extracted.index[-1] , '%Y-%m-%d').date() +  dt.timedelta(days=1)
print(end_date, extracted_date)

2020-08-07 2020-08-08


In [6]:
complete_results = {} 
company_template = {'strat_incdec':{'0-200' : 0, '0-100': 0, '8-21': 0, '50-200': 0},  'strat_smaema' : 0 , 'strat_point5all': {'near_close' : 0, 'above_close' : 0, 'below_close':0}, 'strat_macd' : {'normal_crossover' : 0, 'middle_crossover' : 0, 'hist_turnover': 0}, 'strat_hieken_ashi' :{'Bullish': 0, 'Bearish': 0, 'Breakpoint': 0}, 'strat_fisher': {'Bearish' : 0, 'Bullish': 0, 'Middle_cross': 0}, 'strat_stochrsi': {'Bullish' : 0, 'Bearish': 0, 'Middle_cross': 0} }

In [8]:
with open("nifty500.pickle", "rb") as f:
    tickers = pickle.load(f)
    
for ticker in tqdm(tickers):
    if os.path.exists('stocks_dfs/{}.csv'.format(ticker)):
        main_df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)    
        main_df.set_index('Date', inplace = True)
        if extracted_date <= dt.date.today():
            if ticker == 'NIFTY':
                new_df = get_history(symbol=ticker, start = extracted_date, end = dt.date.today(), index = True)
            else:
                new_df = get_history(symbol=ticker, start = extracted_date, end = dt.date.today())
            #display(main_df.tail(), new_df)
            main_df = main_df.append(new_df)
            #display(main_df.tail())
            #main_df = generate_data(main_df)
            main_df.to_csv('stocks_dfs/{}.csv'.format(ticker))
    
    
        

100%|████████████████████████████████████████████████████████████████████████████████| 502/502 [09:45<00:00,  1.17s/it]


In [6]:
with open("nifty500.pickle", "rb") as f:
    tickers = pickle.load(f)
size_zero = 0
incorrect_data = 0 
for ticker in tqdm(tickers):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    df.index = df.Date
    if df.size == 0:
        print(ticker, 'zero')
        size_zero += 1
        tickers.remove(ticker)
        continue
            
    if df.index[-1] != str(end_date):
        print(ticker, 'incomplete')
        incorrect_data += 1
        tickers.remove(ticker)
        continue
    complete_results[ticker] = copy.deepcopy(company_template)         

 14%|██████████▉                                                                      | 68/502 [00:00<00:05, 80.72it/s]

BANDHANBNK incomplete


 17%|█████████████▌                                                                   | 84/502 [00:01<00:05, 76.71it/s]

BBTC zero


 22%|█████████████████▏                                                              | 108/502 [00:01<00:05, 77.64it/s]

...

 76%|█████████████████████████████████████████████████████████████                   | 383/502 [00:05<00:01, 71.79it/s]

RELCAPITAL incomplete


 79%|███████████████████████████████████████████████████████████████▌                | 399/502 [00:05<00:01, 71.29it/s]

SHREECEM incomplete


 98%|██████████████████████████████████████████████████████████████████████████████▏ | 491/502 [00:06<00:00, 73.79it/s]


In [120]:
def generate_scores(ticker):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    display(ticker)
    df = generate_data(df)
    df['Score'] = None
    df['BSH'] = None
    score_index = df.index[df.Date == '2020-03-23'].tolist()
    for i in tqdm(range((score_index[0]), len(df.index))):
        company_results = copy.deepcopy(company_template)
        today = i
        yesterday = i-1
        yesterday2 = i-2   
        #print('result', runstrat_incdec(df, ticker, today , print_results = False))
        company_results['strat_incdec'] = runstrat_incdec(df, ticker, today , print_results = False)
        #print(df.Date[i], company_results['strat_incdec'])
        company_results['strat_point5all'] = runstrat_point5(df, ticker, today, [0.25, 0.5], ['8ma', '14ma', '21ma'])        
        
        
        
for ticker in (tickers):
    generate_scores(ticker)
    break  
    
    

'3MINDIA'

  3%|██▊                                                                                | 3/89 [00:00<00:03, 27.06it/s]

here
Date                   2020-03-23
Symbol                    3MINDIA
Series                         EQ
Prev Close                17366.3
Open                        16300
High                        16600
Low                         15701
Last                        15818
Close                     15887.2
VWAP                        16039
Volume                      14420
Turnover              2.31283e+13
Trades                       4040
Deliverable Volume          10948
%Deliverble                0.7592
3ma                       16728.2
5ma                         17159
35d                      -2.57497
8ma                       17898.7
58d                      -4.31098
14ma                      19526.2
814d                     -9.09308
21ma                      19964.9
1421d                    -2.24657
50ma                      21451.1
2150d                    -7.44411
100ma                     21629.9
50100d                  -0.833238
200ma                     21457.8
100200d  

  7%|█████▌                                                                             | 6/89 [00:00<00:03, 25.93it/s]


{0.25: {'8ma': [], '14ma': [], '21ma': []}, 0.5: {'8ma': [], '14ma': [], '21ma': []}}
Enter Weights of MAS : 
here
Date                   2020-03-30
Symbol                    3MINDIA
Series                         EQ
Prev Close                17648.6
Open                        17210
High                        17490
Low                         17000
Last                        17480
Close                     17436.3
VWAP                      17325.7
Volume                       2764
Turnover              4.78883e+12
Trades                       1422
Deliverable Volume           1402
%Deliverble                0.5072
3ma                       17674.9
5ma                       17180.7
35d                       2.79607
8ma                         17011
58d                      0.987654
14ma                      17841.2
814d                     -4.88002
21ma                      19046.6
1421d                    -6.75618
50ma                      20965.4
2150d                    -10.0745


...


{0.25: {'8ma': [], '14ma': [], '21ma': []}, 0.5: {'8ma': [], '14ma': [], '21ma': []}}
Enter Weights of MAS : 
here
Date                   2020-07-20
Symbol                    3MINDIA
Series                         EQ
Prev Close                21041.2
Open                        21100
High                        21600
Low                         20887
Last                        21600
Close                     21532.5
VWAP                      21347.9
Volume                       4821
Turnover              1.02918e+13
Trades                       2284
Deliverable Volume           2394
%Deliverble                0.4966
3ma                       21047.8
5ma                       20723.7
35d                       1.53969
8ma                       20319.7
58d                       1.94945
14ma                      19698.7
814d                      3.05649
21ma                      19358.6
1421d                     1.72625
50ma                      18531.7
2150d                     4.27135


 96%|██████████████████████████████████████████████████████████████████████████████▎   | 85/89 [00:04<00:00, 20.05it/s]


{0.25: {'8ma': [], '14ma': [], '21ma': []}, 0.5: {'8ma': [], '14ma': [], '21ma': []}}
Enter Weights of MAS : 
here
Date                  2020-07-27
Symbol                   3MINDIA
Series                        EQ
Prev Close               21492.2
Open                       21360
High                     21582.7
Low                        21160
Last                       21555
Close                    21522.7
VWAP                     21491.1
Volume                      2427
Turnover              5.2159e+12
Trades                      1125
Deliverable Volume          1343
%Deliverble               0.5534
3ma                      21505.9
5ma                      21452.9
35d                     0.246521
8ma                        21301
58d                     0.708121
14ma                     20648.5
814d                     3.06338
21ma                     20045.5
1421d                    2.92026
50ma                     18876.8
2150d                    5.83034
100ma                    1

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [00:04<00:00, 19.37it/s]

Enter Weights of MAS : 
here
Date                  2020-07-30
Symbol                   3MINDIA
Series                        EQ
Prev Close               21501.7
Open                       21450
High                       21900
Low                        20750
Last                     21711.9
Close                    21197.5
VWAP                     21124.2
Volume                      6369
Turnover              1.3454e+13
Trades                      3222
Deliverable Volume          2935
%Deliverble               0.4608
3ma                      21403.8
5ma                      21445.2
35d                     -0.19372
8ma                      21434.5
58d                     0.050151
14ma                     21073.6
814d                     1.68354
21ma                       20412
1421d                    3.13963
50ma                     19132.9
2150d                    6.26625
100ma                    18917.8
50100d                   1.12408
200ma                    20404.5
100200d       

In [53]:
for ticker in tqdm(tickers):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    cols = ['Date', 'Symbol', 'Series', 'Score','Remarks', 'Prev Close', 'Open', 'High',
       'Low', 'Last', 'Close', 'VWAP', 'Volume', 'Turnover', 'Trades',
       'Deliverable Volume', '%Deliverble', '3ma', '5ma', '35d', '8ma', '58d',
       '14ma', '814d', '21ma', '1421d', '50ma', '2150d', '100ma', '50100d',
       '200ma', '100200d', 'ema10']
    #cols = cols[-1:] + cols[:-1]
    df = df[cols]       
    df.to_csv('stocks_dfs/{}.csv'.format(ticker), index = False) 
    
    
    

100%|███████████████████████████████████████████████████████████████████████████████▊| 501/502 [00:17<00:00, 29.00it/s]


KeyError: "['Deliverable Volume', 'Prev Close', 'VWAP', '%Deliverble', 'Symbol', 'Series', 'Trades', 'Last'] not in index"

In [83]:
def inc_dec(df, ticker, today, marange): #["Close", "3ma", "5ma" , "8ma"]
    increasing = {}
    decreasing = {}
    for ma in marange:
        if(ma == 'Close'):
            continue
        increasing[ma] = []
        decreasing[ma] = []
    today_data = df.iloc[today]
    #print(today_data)
    inc = 1
    dec = 1
    for i in range(0, len(marange)-1):
        if(today_data[marange[i]] >= today_data[marange[i+1]] and inc ==1):
            increasing[marange[i+1]].append(ticker)
        else:
            inc = 0
            break

    for i in range(0, len(marange)-1):
        if(today_data[marange[i]] < today_data[marange[i+1]] and dec == 1):
            decreasing[marange[i+1]].append(ticker)
        else:
            dec = 0
            break
    return increasing , decreasing

def runstrat_incdec(df, ticker, today, print_results = False):        
    complete_results[ticker]['strat_incdec'] = copy.deepcopy(company_template['strat_incdec'])
    #get first result
    maranges = ["Close","3ma","5ma", "8ma", "14ma", "21ma", "50ma", "100ma" , "200ma"]
    increasing, decreasing = inc_dec(df, ticker, today, maranges)
    
    for ticker in increasing['200ma']: 
        complete_results[ticker]['strat_incdec']['0-200'] = 5

    for ticker in increasing['100ma']:
        complete_results[ticker]['strat_incdec']['0-100'] = 4

    for ticker in increasing['200ma']:
        complete_results[ticker]['strat_incdec']['0-200'] = -5

    for ticker in increasing['100ma']:
        complete_results[ticker]['strat_incdec']['0-100'] = -4
    
    #get secong results
    maranges = ["Close","8ma", "14ma", "21ma"]
    increasing, decreasing = inc_dec(df, ticker, today, maranges)
    
    for ticker in increasing['21ma']:
        complete_results[ticker]['strat_incdec']['8-21'] = 3

    for ticker in decreasing['21ma']:
        complete_results[ticker]['strat_incdec']['8-21'] = -3
        
    #get third result
    maranges = ["Close","50ma", "100ma", "200ma"]
    increasing, decreasing = inc_dec(df, ticker, today, maranges)
    for ticker in increasing['200ma']:
        complete_results[ticker]['strat_incdec']['50-200'] = 1

    for ticker in decreasing['200ma']:
        complete_results[ticker]['strat_incdec']['50-200'] = -1
        
    return complete_results[ticker]['strat_incdec']

for ticker in tickers[:2]:
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    df = generate_data(df)
    today = df.index[-1]
    runstrat_incdec(df, ticker, today)            
            


In [119]:
strat_point5_results = {}
import math
def deviation(today, ndma, pcts):
    n = len(pcts)
    #[]
    diff = today[ndma] - today['Close']
    dev = ((diff/today['Close'])*100)
    if -pcts[n-1] < dev <= -pcts[n-2]:
        dev = -pcts[n-1]
    elif -pcts[n-2] < dev <= -pcts[n-3]:
        dev = -pcts[n-2]
    elif -pcts[n-3] < dev < -pcts[n-4]:
        dev = -pcts[n-3]
    elif -pcts[n-4] < dev < 0:
        dev = -pcts[n-4]
    elif 0 < dev <= pcts[n-4]:
        dev = pcts[n-4]
    elif pcts[n-4] < dev <= pcts[n-3]:
        dev = pcts[n-3]
    elif pcts[n-3] < dev < pcts[n-2]:
        dev = pcts[n-2]
    elif pcts[n-2] < dev < pcts[n-1]:
        dev = pcts[n-1]
    #print(dev)
    return dev

def mov_avg(df, company,today, pcts, mas):
    movavg_results = {}
    for pct in pcts:
        movavg_results[pct] = {}
        for ma in mas:
            movavg_results[pct][ma] = []

    df = pd.read_csv('stocks_dfs/{}.csv'.format(company), header= 0)
    today_data = (df.iloc[today])
    print(today_data)
    i = 0
    for ma in mas:
        mad = deviation(today_data, ma, pcts)
        if abs(mad) in movavg_results:
            if(mad < 0):
                company1 = "+"+company
            else:
                company1 = "-"+ company
            movavg_results[abs(mad)][mas[i]].append(company1)
        i+=1 
    print(movavg_results)
    return movavg_results   

In [117]:
strat_point5all_results = {'near_close':[], 'above_close': [], 'below_close':[]}

def strat_point5all_close(df, company ,today, pcts, mas, show_table = False):
    print('here')
    movavg_results = mov_avg(df, company, today, pcts, mas)
    companys = []
    print("Enter Weights of MAS : ")
    weights = []
    for i in range(0,len(mas)):
        #wt = int(input('Weight for {}:'.format(mas[i])))
        wt = 1
        weights.append(wt)

    for i in pcts:
        for j in mas:
            for c in movavg_results[i][j]:
                companys.append(c[1:])

    companys = list(set(companys))    
    range_table = {'Company':[],'cscore':[], 'Overview':[]}
    for pct in pcts:
        name = 'RANGE : -{}-+{}'.format(pct, pct)
        range_table[name] = []
        
    key = list(range_table.keys())
    
    for company in companys:
        newcomp = []
        occourence = []
        score = 0
        pos_score = 0
        neg_score = 0
        for i in range(0, len(pcts)):
            newstr = ""
            for j  in range(0, len(mas)):
                if '+'+company in movavg_results[pcts[i]][mas[j]]:
                    newstr = newstr+',     '+ '+' +mas[j]
                    occourence.append(mas[j])
                    pos_score =pos_score+ (len(pcts)-i)*weights[j]
                if '-'+company  in movavg_results[pcts[i]][mas[j]]:
                    newstr = newstr+',     '+ '-'+mas[j]
                    occourence.append(mas[j])
                    neg_score =neg_score+ (len(pcts)-i)*weights[j]
            newcomp.append(newstr[2:])
        
        overview = "+{} | -{}".format(pos_score, neg_score)
        newcomp.insert(0,overview)
        score = abs(pos_score+neg_score)
        newcomp.insert(0, score)
        newcomp.insert(0, company)
        if(len(occourence) < len(mas)):
            continue
        for i in range(0, len(newcomp)):
            range_table[key[i]].append(newcomp[i])   
    table = pd.DataFrame.from_dict(range_table)
    table = table.set_index('Company')
    table = table.sort_values(by='cscore', ascending=False)
    table = table.drop(columns = ['cscore'])
    if show_table :
        display(table)
    table.to_csv('maranges_81421.csv')
    return table
'''
pcts = [0.25, 0.5]
mas = ['8ma', '14ma', '21ma']
point5all_table = strat_point5all_close(pcts, mas, tickers, show_table = False)
point5all_companys = list(point5all_table.index)
for company in point5all_companys:
    strat_point5all_results['near_close'].append(company)'''

def strat_point5all_anywhere(pct, mas, show_table = False):
    companys = []
    for company in tickers:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(company), header= 0)
        today = (df.iloc[df.shape[0]-1])
        values = []
        for ma in mas:
            values.append(today[ma])
        max_val = max(values)
        min_val = min(values)
        mid = (max_val + min_val)/2
        if( (max_val - mid)/mid < (pct/200)  and (mid - min_val)/mid < (pct/200) ):
            #print(company ,((max_val - mid)/mid)*100, ((mid - min_val)/mid)*100)
            if(min_val > today.Close):
                strat_point5all_results['above_close'].append(company)
            if(max_val < today.Close):
                strat_point5all_results['below_close'].append(company)                
            companys.append(company)
    if(show_table):
        display(companys)
    return companys

'''mas = ["8ma", "14ma" , "21ma"]
companys = strat_point5all_anywhere(0.5, mas, show_table = False)
print("8/14/21 MAS near each other anywhere irrespective of trade value")
for company in strat_point5all_results['near_close']:
    complete_results[company]['strat_point5all']['near_close'] = 3

for company in strat_point5all_results['above_close']:
    complete_results[company]['strat_point5all']['above_close'] = 2    

for company in strat_point5all_results['below_close']:
    complete_results[company]['strat_point5all']['below_close'] = 2        '''
    
#print(strat_point5all_results)
def runstrat_point5(df, company, today, pct, mas):
    strat_point5all_close(df, company , today, pct, mas, show_table = False)
    #strat_point5all_anywhere(df, company, today, pct[-1], mas, show_table = False)
    

In [ ]:
strat_sma_ema_cross_results = {'Bullish' : [], 'Bearish': []}
def runstrat_sma_ema_cross():
    cross = []
    for ticker in tickers:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        df['ema10'] = df['Close'].ewm(span=10).mean()
        df['10ma'] = df['Close'].rolling(window = 10, min_periods = 0).mean()
        today = df.iloc[df.shape[0]-1]
        todaysma = (df.iloc[df.shape[0]-1]['10ma'])
        todayema = (df.iloc[df.shape[0]-1]['ema10'])
        yesterdaysma = (df.iloc[df.shape[0]-2]['10ma'])
        yesterdayema = (df.iloc[df.shape[0]-2]['ema10'])
        tdiff = todayema - todaysma
        ydiff = yesterdayema - yesterdaysma
        if(ydiff*tdiff < 0):
            if tdiff > 0:
                cross.append("+"+ticker)
                if(todaysma > today.Close):
                    complete_results[ticker]['strat_smaema'] = 2
                elif 1.1*today.Close> todaysma > 0.9*today.Close:
                    complete_results[ticker]['strat_smaema'] = 2
                else:
                    complete_results[ticker]['strat_smaema'] = 2
                strat_sma_ema_cross_results['Bullish'].append(ticker)
            else:
                cross.append("-"+ticker)
                if(todaysma < today.Close):
                    complete_results[ticker]['strat_smaema'] = -2
                elif 1.1*today.Close> todaysma > 0.9*today.Close:
                    complete_results[ticker]['strat_smaema'] = -2
                else:
                    complete_results[ticker]['strat_smaema'] = -2
                strat_sma_ema_cross_results['Bearish'].append(ticker)
    return cross
        
cross = runstrat_sma_ema_cross()
print("SMA|EMA CROSSOVERS")
print(cross)   

In [ ]:
strat_macd_results = {'bullish_crossover' : [], 'bearish_crossover': [], 'middle_crossover_pos': [], 'middle_crossover_neg': [], 'hist_bearish_turn': [], 'hist_bullish_turn': []}
def macd_study():
    for ticker in tickers:        
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        macd = exp1-exp2
        signal = macd.ewm(span=9, adjust=False).mean()
        df['hist'] = macd-signal
        hist = TA.SMA(df, 3, "hist")
        today_hist = hist[hist.shape[0]-1]
        yesterday_hist1 = hist[hist.shape[0]-2]
        yesterday_hist2 = hist[hist.shape[0]-3]
        today_macd = macd[macd.shape[0]-1]
        yesterday_macd = macd[macd.shape[0]-2]
        today_signal = signal[signal.shape[0]-1]
        yesterday_signal = signal[signal.shape[0]-2]
        tdiff = today_macd - today_signal
        ydiff = yesterday_macd - yesterday_signal
        if(today_hist > 0 and yesterday_hist2 < yesterday_hist1 and yesterday_hist1 > today_hist):
            strat_macd_results['hist_bearish_turn'].append(ticker)
        if(today_hist < 0 and yesterday_hist2 > yesterday_hist1 and yesterday_hist1 < today_hist):
            strat_macd_results['hist_bullish_turn'].append(ticker)
        if tdiff*ydiff < 0:
            if tdiff > 0 :
                strat_macd_results['bullish_crossover'].append(ticker)
            if tdiff < 0 :
                strat_macd_results['bearish_crossover'].append(ticker)
        if(today_macd < 0 and yesterday_macd > 0):
            strat_macd_results['middle_crossover_neg'].append(ticker)
        elif(today_macd > 0 and yesterday_macd < 0):
            strat_macd_results['middle_crossover_pos'].append(ticker)                
        
            
print("MACD NEAR 0")  

macd_study()
for company in strat_macd_results['bullish_crossover']:
    complete_results[company]['strat_macd']['normal_crossover'] =2
for company in strat_macd_results['bearish_crossover']:
    complete_results[company]['strat_macd']['normal_crossover'] = -2
for company in strat_macd_results['middle_crossover_neg']:
    complete_results[company]['strat_macd']['middle_crossover'] = -3
for company in strat_macd_results['middle_crossover_pos']:
    complete_results[company]['strat_macd']['middle_crossover'] = 3
for company in strat_macd_results['hist_bullish_turn']:
    complete_results[company]['strat_macd']['hist_turnover'] = 1
for company in strat_macd_results['hist_bearish_turn']:
    complete_results[company]['strat_macd']['hist_turnover'] = -1
print(strat_macd_results)

In [ ]:
def run_strat_RSI(series, period=14, show_results = False):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    x = 100 - 100 / (1 + rs)
    return x 

for ticker in tickers:
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)        
    x = run_strat_RSI(df.Close, 14)
    break;

In [ ]:
strat_stochrsi_results = {'negcrossover':  [], 'poscrossover' : [], 'negmiddle' : [], 'posmiddle' : []}
stochrsi_K = []
def run_strat_stochrsi(ticker):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)        
    df = df.rename(columns = {"Close":"close", "Open" : "open", "High" : "high", "Low": "low", "Volume": "volume"}) 
    rsi = run_strat_RSI(df.close, 14)
    stochrsi  = (rsi - rsi.rolling(14).min()) / (rsi.rolling(14).max() - rsi.rolling(14).min())
    stochrsi_K = stochrsi.rolling(3).mean()
    stochrsi_D = stochrsi_K.rolling(3).mean()
    todayK = stochrsi_K[len(stochrsi_K) -1+14]
    todayD = stochrsi_D[len(stochrsi_D) -1+14]
    yesterdayK = stochrsi_K[len(stochrsi_K) -2+14]
    yesterdayD = stochrsi_D[len(stochrsi_D) -2+14]
    tdiff = todayK - todayD
    ydiff = yesterdayK - yesterdayD
    
    if abs(todayK - todayD)/min(todayK, todayD) < 0.1 and 0.16<todayK<0.24 :
        strat_stochrsi_results['poscrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bullish'] = 3
    elif abs(todayK - todayD)/min(todayK, todayD) < 0.1 and 0.76<todayK<0.84 :
        strat_stochrsi_results['negcrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bearish'] = -3
    elif 0.16<todayK<0.2 and yesterdayK<todayK:
        strat_stochrsi_results['poscrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bullish'] = 2
    elif 0.76<todayK<0.80 and yesterdayK>todayK:
        strat_stochrsi_results['negcrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bearish'] = -2       
        
        
        
    if(ydiff*tdiff < 0):
        if tdiff < 0 and yesterdayK > 0.8:
            strat_stochrsi_results['negcrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bearish'] = -3
        elif tdiff > 0 and yesterdayK > 0.8:
            strat_stochrsi_results['negcrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bullish'] = 1
        elif tdiff < 0 and yesterdayK < 0.8:
            strat_stochrsi_results['negmiddle'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Middle_cross'] = -1
        elif tdiff > 0 and yesterdayK < 0.2:
            strat_stochrsi_results['poscrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bullish'] = 3
        elif tdiff < 0 and yesterdayK < 0.2:
            strat_stochrsi_results['poscrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bullish'] = -1
        elif tdiff > 0 and yesterdayK > 0.2:
            strat_stochrsi_results['posmiddle'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Middle_cross'] = 1
    
for ticker in tickers:  
    run_strat_stochrsi(ticker)
print(strat_stochrsi_results)

In [ ]:
strat_hieken_ashi_results = {"SuperBullish" : [], "SuperBearish" : [], "Bullish" : [] , "Bearish": [], "Breakpoint" : []}

def heiken_ashi(ticker):    
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)    
    df['HA_Close']=(df['Open']+ df['High']+ df['Low']+df['Close'])/4

    idx = df.index.name
    df.reset_index(inplace=True)

    for i in range(0, len(df)):
        if i == 0:
            df.at[i, 'HA_Open'] = ((df.at[i, 'Open'] + df.at[i, 'Close']) / 2)
        else:
            df.at[i, 'HA_Open'] =  ((df.at[i - 1, 'HA_Open'] + df.at[i - 1, 'HA_Close']) / 2)

    if idx:
        df.set_index(idx, inplace=True)
    df['HA_High']=df[['HA_Open','HA_Close','High']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','Low']].min(axis=1)
    df = df.iloc[df.shape[0]-1]
    if(df['HA_Low'] == df['HA_Open'] and df['HA_Close'] >= df['HA_Open'] + df['HA_Open']*0.04):
        strat_hieken_ashi_results["SuperBullish"].append(ticker)
    if(df['HA_Low'] == df['HA_Open'] and df['HA_Open'] + df['HA_Open']*0.02 <= df['HA_Close'] <= df['HA_Open'] + df['HA_Open']*0.04):
        strat_hieken_ashi_results["Bullish"].append(ticker)
    if(df['HA_High'] == df['HA_Open'] and df['HA_Close'] <= df['HA_Open'] - df['HA_Open']*0.04):
        strat_hieken_ashi_results["SuperBearish"].append(ticker)
    if(df['HA_High'] == df['HA_Open'] and df['HA_Open'] - df['HA_Open']*0.04 < df['HA_Close'] <= df['HA_Open'] - df['HA_Open']*0.02):
        strat_hieken_ashi_results["Bearish"].append(ticker)
    body = abs(df['HA_Open'] - df['HA_Close'])
    if(df['HA_High'] >= df['HA_Open'] + body and df['HA_Low'] <= df['HA_Open'] - body):
        strat_hieken_ashi_results["Breakpoint"].append(ticker)    
        

for company in tickers:
    heiken_ashi(company)

for x in strat_hieken_ashi_results:
    for ticker in strat_hieken_ashi_results[x]:
        if x == 'SuperBullish':
            complete_results[ticker]['strat_hieken_ashi']['Bullish'] = 3
        elif x == 'Bullish':
            complete_results[ticker]['strat_hieken_ashi']['Bullish'] = 2
        elif x == 'SuperBearish':
            complete_results[ticker]['strat_hieken_ashi']['Bearish'] = -3
        elif x == 'Bearish':
            complete_results[ticker]['strat_hieken_ashi']['Bearish'] = -2
        elif x == 'Breakpoint':
            complete_results[ticker]['strat_hieken_ashi']['Breakpoint'] = 4
    

In [ ]:
display(strat_hieken_ashi_results)

In [ ]:
strat_fisher_results = {'negcrossover': [], 'poscrossover': [],'neartopline':[] , 'nearbotline': [], 'negmiddle':[], 'posmiddle': [] }
def fish(tickers):
    negcrossover = []
    poscrossover = []
    neartopline = []
    nearbotline = []
    negmiddle = []
    posmiddle = []
    for ticker in tickers:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        df = df.rename(columns = {"Close":"close", "Open" : "open", "High" : "high", "Low": "low", "Volume": "volume"}) 
        #idx = df.index.name
        #df.reset_index(inplace=True)
        df['fish'] = TA.FISH(df, 10)
        #df['fish'] = TA.SMA(df, 3, "fish")    
        today = df.iloc[df.shape[0]-1]['fish']
        yesterday = df.iloc[df.shape[0]-2]['fish']
        yesterday2 = df.iloc[df.shape[0]-3]['fish']
        if(yesterday2 < yesterday and yesterday >= today):
            if(today > 1.45):
                negcrossover.append(ticker)
            elif  -1.45 < today < 1.45:
                negmiddle.append(ticker)
        if(yesterday2 > yesterday and yesterday <= today):
            if(today < -1.45):
                poscrossover.append(ticker)
            elif  -1.45 < today < 1.45:
                posmiddle.append(ticker)

        if(yesterday2 > yesterday and  1.55 > today > 1.45 ):
            neartopline.append(ticker)
        if(yesterday2 < yesterday and  -1.55 < today < -1.45 ):
            nearbotline.append(ticker) 
    strat_fisher_results['negcrossover']= negcrossover
    strat_fisher_results['poscrossover']= poscrossover
    strat_fisher_results['neartopline']= neartopline
    strat_fisher_results['nearbotline']= nearbotline
    strat_fisher_results['negmiddle']= negmiddle
    strat_fisher_results['posmiddle']= posmiddle
    
fish(tickers)
for x in strat_fisher_results:
    for ticker in strat_fisher_results[x]:
        if x == 'poscrossover':
            complete_results[ticker]['strat_fisher']['Bullish'] = 2
        elif x == 'negcrossover':
            complete_results[ticker]['strat_fisher']['Bearish'] = -2
        elif x == 'neartopline':
            complete_results[ticker]['strat_fisher']['Bearish'] = -3
        elif x == 'nearbotline':
            complete_results[ticker]['strat_fisher']['Bullish'] = 3
        elif x == 'negmiddle':
            complete_results[ticker]['strat_fisher']['Middle_cross'] = -1
        elif x == 'posmiddle':
            complete_results[ticker]['strat_fisher']['Middle_cross'] = 1
            
fishpos = set(strat_fisher_results['poscrossover'])|set(strat_fisher_results['nearbotline'])|set(strat_fisher_results['posmiddle'])
fishneg = set(strat_fisher_results['negcrossover'])|set(strat_fisher_results['neartopline'])|set(strat_fisher_results['negmiddle'])

In [ ]:
display(strat_fisher_results)

In [ ]:
strat_delivery_results = {}
def strat_delivery():
    for ticker in tickers[:-1]:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        deliverable_5 = df['%Deliverble'].rolling(window=5, min_periods=0).mean().iloc[-1]
        Volume_5 = df['Volume'].rolling(window=5, min_periods=0).mean().iloc[-1]
        today = df.iloc[-1]
        if today['Volume'] > Volume_5*2 or today['%Deliverble'] > deliverable_5*1.5:
            strat_delivery_results[ticker]= {'pricechange': 0, 'Remarks' : ""}
            if today['Volume'] > Volume_5*3:
                strat_delivery_results[ticker]['Remarks'] += "3 times Volume  "
            elif today['Volume'] > Volume_5*2:
                strat_delivery_results[ticker]['Remarks'] += "2 times Volume  "
            if today['%Deliverble'] > deliverable_5*2:
                strat_delivery_results[ticker]['Remarks'] += "2 times Delivery"
            elif today['%Deliverble'] > deliverable_5*1.5:
                strat_delivery_results[ticker]['Remarks'] += "1.5 times Delivery"
            change_cost = (df.iloc[-1]['Close'] - df.iloc[-2]['Close'])/df.iloc[-2]['Close']
            strat_delivery_results[ticker]['pricechange'] = round(change_cost*100,2)
            print(ticker, change_cost*100)
        
strat_delivery()
display(strat_delivery_results)

In [ ]:
mov_avg_table = strat_point5all_close([0.25, 0.5], ["8ma", "14ma", "21ma"], tickers) #mov_avg_81421_near_close
companys_inpoint5_range_anywhere = strat_point5all_anywhere(0.5, ["8ma", "14ma", "21ma"]) #mov_avg_81421_anywhere
A_intersection_B = (set(mov_avg_table.index) & (set(companys_inpoint5_range_anywhere))) #A_intersection_B
A_union_B = (set(mov_avg_table.index) | (set(companys_inpoint5_range_anywhere))) #A_union_B

#SMA/EMA10 CROSSOVER
sma_ema = runstrat_sma_ema_cross()
c1 = []
c2 = []
for company in sma_ema:
    if company[0] == '+':
        c1.append(company[1:]) #(+ve crossover sma10/ema10)
    else:
        c2.append(company[1:]) #(-ve crossover sma10/ema10)

#increasing/decreasing        
inc_dec(["Close","3ma", "5ma", "8ma", "14ma", "21ma", "50ma", "100ma", "200ma"])        
x1 = strat_inc_dec_results['increasing']["21ma"]
y1 = strat_inc_dec_results['decreasing']["21ma"]
x2 = strat_inc_dec_results['increasing']["50ma"]
y2 = strat_inc_dec_results['decreasing']["50ma"]
x3 = strat_inc_dec_results['increasing']["100ma"]
y3 = strat_inc_dec_results['decreasing']["100ma"]
x4 = strat_inc_dec_results['increasing']["200ma"]
y4= strat_inc_dec_results['decreasing']["200ma"]
inc_dec(["Close", "8ma", "14ma", "21ma"])
x5 = strat_inc_dec_results['increasing']["21ma"]
y5 = strat_inc_dec_results['decreasing']["21ma"]
inc_dec(["Close", "50ma", "100ma", "200ma"])
x6 = strat_inc_dec_results['increasing']["200ma"]
y6 = strat_inc_dec_results['decreasing']["200ma"]

#intersecion/union of incdec and SMA/EMA crossover

In [ ]:
p16 = set(c1)&set(x6)
q16 = set(c2)&set(y6)
p13 = set(c1)&set(x3)
q13 = set(c2)&set(y3)
p15 = set(c1)&set(x5)
q15 = set(c2)&set(y5)
p13 = set(c1)&set(x3)
q13 = set(c2)&set(y3) 

super_bullish_smaema_incdec = ((set(x6)|set(x5)|set(x3)) & set(c1))|set(x4)
super_bearish_smaema_incdec = ((set(y6)|set(y5)|set(y3)) & set(c2))|set(y4)
very_bullish_smaema_incdec = ((set(x6)|set(x5)) & set(c1))|set(x3)
very_bearish_smaema_incdec = ((set(y6)|set(y5)) & set(c2))|set(y3)
bullish_smaema_incdec = ((set(x6)&set(x5)) | set(c1))|set(x3)
bearish_smaema_incdec= ((set(y6)&set(y5)) | set(c2))|set(y3)
fish_ints_incdec_pos = (set(x3)|set(x5)|set(x6))& fishpos
fish_ints_incdec_neg = (set(x3)|set(y5)|set(y6))& fishneg
A_ints_B_ints_fish = A_union_B & (fishpos | fishneg)

heikenashi_smaema_incdec_superbullish_pos = set(strat_hieken_ashi_results['SuperBullish']) & set(super_bullish_smaema_incdec)
heikenashi_smaema_incdec_verybullish_pos = set(strat_hieken_ashi_results['Bullish']) & set(very_bullish_smaema_incdec)
heikenashi_smaema_incdec_bullish_pos = set(strat_hieken_ashi_results['Bullish']) & set(bullish_smaema_incdec)
heikenashi_smaema_incdec_superbearish_pos = set(strat_hieken_ashi_results['SuperBearish']) & set(super_bearish_smaema_incdec)
heikenashi_smaema_incdec_verybearish_pos = set(strat_hieken_ashi_results['Bearish']) & set(very_bearish_smaema_incdec)
heikenashi_smaema_incdec_bearish_pos = set(strat_hieken_ashi_results['Bearish']) & set(bearish_smaema_incdec)
heikenashi_breakpoints_smaema_icndec_allbullish = set(strat_hieken_ashi_results['Breakpoint']) & (set(super_bullish_smaema_incdec)|set(very_bullish_smaema_incdec)|set(bullish_smaema_incdec))
heikenashi_breakpoints_smaema_icndec_allbearish = set(strat_hieken_ashi_results['Breakpoint']) & (set(super_bearish_smaema_incdec)|set(very_bearish_smaema_incdec)|set(bearish_smaema_incdec))
fish_heikenashi_pos = set(fishpos)&set(strat_hieken_ashi_results['Bullish'])
fish_heikenashi_neg = set(fishneg)&set(strat_hieken_ashi_results['Bearish'])

In [ ]:
print( "\nA(mov_avg_81421_near_close) ==>> ",list(mov_avg_table.index))
print("\nB(mov_avg_81421_anywhere) ==>> ", companys_inpoint5_range_anywhere)
print("\nA_intersection_B ==> ", A_intersection_B)
print("\nA_union_B ==>> ", A_union_B)
print("\nc1(+ve crossover sma10/ema10) ==>> " , c1 )
print("\nc2(-ve crossover sma10/ema10) ==>> " , c2 )
print("\nSMA/EMA intersection inc/dec super_bullish ==>> (increasing) ", len(super_bullish_smaema_incdec),  super_bullish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec super_bearish==>> (decreasing) ", len(super_bearish_smaema_incdec),  super_bearish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec very_bullish==>> (increasing) ", len(very_bullish_smaema_incdec),  very_bullish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec verybullish==>> (decreasing) ", len(very_bearish_smaema_incdec),  very_bearish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec bullish ==>> (increasing) ", len(bullish_smaema_incdec),  bullish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec bullish ==>> (decreasing) ", len(bearish_smaema_incdec),  bearish_smaema_incdec )
print("\n Fisher incdec results positive ==>> ", len(fish_ints_incdec_pos), fish_ints_incdec_pos)
print("\n Fisher incdec results negative ==>> ", len(fish_ints_incdec_neg), fish_ints_incdec_neg)
print("\n Fisher ints 0.5 results ==>> ", len(A_ints_B_ints_fish), A_ints_B_ints_fish)

print("\n heikenashi_smaema_incdec_superbullish_pos ==>> ", len(heikenashi_smaema_incdec_superbullish_pos) , heikenashi_smaema_incdec_superbullish_pos)      
print("\n heikenashi_smaema_incdec_superbearish_pos ==>> ", len(heikenashi_smaema_incdec_superbearish_pos) , heikenashi_smaema_incdec_superbearish_pos)      

print("\n heikenashi_smaema_incdec_verybullish_pos ==>> ", len(heikenashi_smaema_incdec_verybullish_pos) , heikenashi_smaema_incdec_verybullish_pos)      
print("\n heikenashi_smaema_incdec_verybearish_pos ==>> ", len(heikenashi_smaema_incdec_verybearish_pos) , heikenashi_smaema_incdec_verybearish_pos)      

print("\n heikenashi_smaema_incdec_bullish_pos ==>> ", len(heikenashi_smaema_incdec_bullish_pos) , heikenashi_smaema_incdec_bullish_pos)      
print("\n heikenashi_smaema_incdec_bearish_pos ==>> ", len(heikenashi_smaema_incdec_bearish_pos) , heikenashi_smaema_incdec_bearish_pos)      

print("\n heikenashi_breakpoints_smaema_icndec_allbullish ==>> ", len(heikenashi_breakpoints_smaema_icndec_allbullish) , heikenashi_breakpoints_smaema_icndec_allbullish)      
print("\n heikenashi_breakpoints_smaema_icndec_allbearish ==>> ", len(heikenashi_breakpoints_smaema_icndec_allbearish) , heikenashi_breakpoints_smaema_icndec_allbearish)      

print("\nfish_heikenashi_pos ==>> ",len(fish_heikenashi_pos), fish_heikenashi_pos)
print("\nfish_heikenashi_neg ==>> ",len(fish_heikenashi_neg), fish_heikenashi_neg)



In [ ]:
table_results = {}
table_results['CScore'] = []
table_results['Company'] = []
for strat in complete_results['HDFCBANK']:
    print(strat)
    if strat == 'strat_smaema':
        table_results[strat] = []
        continue
    for category in complete_results['HDFCBANK'][strat]:
        name = strat+'('+category+')'
        table_results[name] = []
print(table_results)                 
print(table_results['strat_incdec(0-200)'])     

for company in complete_results:
    table_results['Company'].append(company)
    score = 0   
    pos_score = 0
    neg_score = 0
    for strat in complete_results[company]:
        if strat == 'strat_smaema':
            table_results[strat].append(complete_results[company][strat])
            score += complete_results[company][strat]
            if complete_results[company][strat] < 0:
                neg_score += complete_results[company][strat]
            else:
                pos_score += complete_results[company][strat]                
            continue
        #print(company, strat , complete_results[company][strat])
        for category in complete_results[company][strat]:
            name = strat+'('+category+')'
            #print(company, strat, name, table_results[name])
            table_results[name].append(complete_results[company][strat][category])   
            score += complete_results[company][strat][category]
            if complete_results[company][strat][category] < 0:
                neg_score += complete_results[company][strat][category]
            else:
                pos_score += complete_results[company][strat][category]  
    table_results['CScore'].append(score)

table = pd.DataFrame.from_dict(table_results)
table = table.set_index('Company')
table = table.sort_values(by='CScore', ascending=False)
name = "Results_" + str(dt.date.today())+ ".csv"
table.to_csv(name)
display(table)


In [ ]:
table_results = {}
table_results['CScore'] = []
table_results['VOLUME/DELIVERY REMARKS'] = []
table_results['Sector'] = []
table_results['Score'] = []
table_results['Pos_Score'] = []
table_results['Neg_Score'] = []
table_results['Pos_Strat'] = []
table_results['Neg_Strat'] = []
table_results['Company'] = []
for strat in complete_results['HDFCBANK']:
    if strat == 'strat_smaema':
        table_results[strat] = []
        continue
    for category in complete_results['HDFCBANK'][strat]:
        name = strat+'('+category+')'
        table_results[name] = []

for company in complete_results:
    if company in strat_delivery_results:
        table_results['VOLUME/DELIVERY REMARKS'].append(strat_delivery_results[company]['Remarks'] + str(strat_delivery_results[company]['pricechange']))
    else: 
        table_results['VOLUME/DELIVERY REMARKS'].append("")
    if company in complete_results:
        table_results['Company'].append(company)
    if company == 'NIFTY':
        table_results['Sector'].append('-')
    else:
        table_results['Sector'].append(Sector[company])
    score = 0  
    cscore = 0
    pos_score = 0
    neg_score = 0
    pos_strat = 0
    neg_strat = 0
    for strat in complete_results[company]:
        if strat == 'strat_smaema':
            table_results[strat].append(complete_results[company][strat])
            score += complete_results[company][strat]
            cscore += abs(complete_results[company][strat])
            if complete_results[company][strat] < 0:
                neg_score += complete_results[company][strat]
                neg_strat += 1
            elif  complete_results[company][strat] >0:
                pos_score += complete_results[company][strat] 
                pos_strat += 1
            continue
        #print(company, strat , complete_results[company][strat])
        for category in complete_results[company][strat]:
            name = strat+'('+category+')'
            #print(company, strat, name, table_results[name])
            table_results[name].append(complete_results[company][strat][category])   
            score += complete_results[company][strat][category]
            cscore += abs(complete_results[company][strat][category])
            if complete_results[company][strat][category] < 0:
                neg_score += complete_results[company][strat][category]
                neg_strat += 1
            elif  complete_results[company][strat][category] >0:
                pos_score += complete_results[company][strat][category]
                pos_strat += 1
    table_results['Pos_Score'].append(pos_score)
    table_results['Neg_Score'].append(neg_score)
    table_results['Pos_Strat'].append(pos_strat)    
    table_results['Neg_Strat'].append(neg_strat)    
    table_results['CScore'].append('+' + str(pos_score) + ' | -' + str(neg_score))
    table_results['Score'].append(cscore)

table = pd.DataFrame.from_dict(table_results)
table = table.set_index('Company')
table = table.sort_values(by='Score', ascending=False)
table['Score'] = table['Pos_Score'] + table['Neg_Score']
table = table.rename(columns = {"CScore":" Net Score: High :  Bullish Low: Bearish midlle : Hold ",
                                'strat_incdec(0-200)' : "INC/DEC 0 to 200 days SMA increasing order (+5) and decreasing order    (-5 )",
                                "strat_incdec(0-100)" : "INC/DEC 0 to 100 days SMA increasing order (+4) and decreasing order (-4 )",
                                "strat_incdec(8-21)": "INC/DEC 8, 14, 21  days SMA increasing order (+3) and decreasing order (-3 )",
                                "strat_incdec(50-200)": "INC/DEC 50, 100, 200  days SMA increasing order (+1) and decreasing order (-1 )",
                                "strat_smaema" : "SMA/EMA CrossOvers",
                                'strat_point5all(near_close)': "POINT5 SMA are near 0.5% of close (+2)",
                                "strat_point5all(above_close)":"POINT5 SMA are in  0.5% of of each other but above close (-1)",
                                "strat_point5all(below_close)" : "POINT5 SMA are in  0.5% of of each other but above close (+1)",
                                "strat_macd(normal_crossover)" : "MACD change : positve gradient to negative gardient (+2) and vice versa (-2) ",                               
                                'strat_macd(middle_crossover)': "MACD crosees ZERO line : positive (+3) and negative   (-3) ",
                                'strat_macd(hist_turnover)': "MACD Histog. change: positve gradient to negative gardient (+1) and vice versa (-1)  ",                                
                                'strat_hieken_ashi(Bearish)': "HELKIN ASHI Negative candle :  if 4% or 2% of CL candle body the -3 or -2",                                
                                'strat_fisher(Bearish)': "FISHER Overbought zone cross over : positive crossover (+1) and negative cross over (-3)",
                                'strat_fisher(Bullish)': "FISHER Oversold zone cross over : positive crossover (+3) and negative cross over (-1)",
                                'strat_fisher(Middle_cross)': 'FISHER Middle zone cross over : positive crossover (+2) and negative cross over (-2)',
                                'strat_stochrsi(Bullish)': 'SRSI Oversold zone cross over : positive crossover (+2) and negative cross over (-1)',
                                'strat_stochrsi(Bearish)': 'SRSI Overbought zone cross over : positive crossover (+1) and negative cross over (-2)',
                                'strat_stochrsi(Middle_cross)': 'SRSI Middle zone cross over : positive crossover (+1) and negative cross over (-1)',                        
                                'strat_hieken_ashi(Bullish)': "HELKIN ASHI Positive candle helkinashi :  if 4% or 2% of CL candle body the +3 or +2",
                                'strat_hieken_ashi(Breakpoint)': 'HELKIN ASHI Change over hieken_ashi candle : (hand and leg shall be 1% of candle body (+4)',
                               }) 
name = "Results_" + str(dt.date.today())+ ".csv"
table.to_csv(name)
display(table.columns)

In [ ]:
display(complete_results['HDFCBANK'])
print(strat_stochrsi_results)

In [ ]:
df = pd.read_csv('stocks_dfs/{}.csv'.format('CONCOR'), header= 0)
exp1 = df['Close'].ewm(span=12, adjust=False).mean()
exp2 = df['Close'].ewm(span=26, adjust=False).mean()
macd = exp1-exp2
signal = macd.ewm(span=9, adjust=False).mean()
df['hist'] = macd-signal
hist = TA.SMA(df, 3, "hist")
print(df['hist'])
print(hist)


In [ ]:
pctchange = {'Company': [],'Sector':[], 'feb': [], 'march':[], 'dip': [], 'today':[], 'remaining':[], 'recovered':[], 'order' : [] }
import math
for ticker in tickers:
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    df.index = df.Date 
    if df.index[-1] != str(dt.date.today()):
        continue
    if '2020-02-24'in df.index:
        if '2020-03-24' in df.index:
            inside = 0 
            feb = df.loc['2020-02-24']['Close']
            march = df.loc['2020-03-24']['Close']
            today = df.iloc[-1]['Close']
            if march > feb :
                dip = ((march-feb)/feb)*100
                recovered = 0
                remaining = 0
                inside = 1
            elif today > feb:
                dip = ((march-feb)/feb)*100
                recovered = 100+((today-feb)/feb)*100
                remaining = 0
                inside = 1
            elif march < today < feb:
                dip = ((march-feb)/feb)*100
                recovered = ((today-march)/feb)*100
                remaining = ((today-feb)/feb)*100
                inside = 1
            else :
                dip = ((march-feb)/feb)*100
                recovered = 0
                remaining = ((today - feb)/feb)*100 -100
                inside = 1
                
            if inside == 0:
                print(ticker)
            pctchange['Company'].append(ticker)
            if ticker == 'NIFTY':
                pctchange['Sector'].append('-')
            else :
                pctchange['Sector'].append(Sector[ticker])
            pctchange['dip'].append(math.ceil(dip))
            pctchange['today'].append(math.ceil(today))
            pctchange['feb'].append(math.ceil(feb))            
            pctchange['march'].append(math.ceil(march))            
            pctchange['recovered'].append(math.ceil(recovered))
            pctchange['remaining'].append(math.ceil(remaining))  
            pctchange['order'].append(math.ceil(remaining * dip))  

            #print(feb, march, today, dip, recovered, remaining)  
    
       


In [ ]:
for i in pctchange:
    print(i, len(pctchange[i]))
table = pd.DataFrame.from_dict(pctchange)
table = table.set_index('Company')    
table = table.sort_values(by=['Sector', 'order', 'recovered'], ascending=False)
table.to_csv('pctchange.csv') 
print(str(dt.date.today()))
display(table)